In [1]:
import FundamentalAnalysis as fa

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from sklearn.preprocessing import MinMaxScaler

In [90]:
# log 정규화
def nplog(dataframe):
    
    temp = np.log(dataframe)
    
    return temp

# indicator 계산
def get_indicators(dataframe, *indicators):
    
    from Indicator import get_Bolinger_Band
    from Indicator import get_DMI
    from Indicator import get_MACD
    from Indicator import get_RSI
    
    

    BAND = get_Bolinger_Band.fnBolingerBand
    DMI = get_DMI.cal_dmi
    MACD = get_MACD.fnMACD
    RSI = get_RSI.fnRSI
    
    
    for indicator in indicators:
        
        if indicator == 'DMI':
            DMI(dataframe)
        elif indicator == 'MACD':
            MACD(dataframe)
        elif indicator == 'RSI':
            RSI(dataframe)
        elif indicator == 'BAND':
            BAND(dataframe)
    
    dataframe.dropna(inplace=True)
    
    return dataframe

# minmax scale
def make_sc_col_df(train_scdf):
    for i in train_scdf.columns:
        globals()['scaler_{}'.format(i)] = MinMaxScaler()
        print("df_{}".format(i))
        globals()["df_{}".format(i)] = train_scdf[[i]]
        globals()["df_{}".format(i)] = globals()['scaler_{}'.format(i)].fit_transform(globals()["df_{}".format(i)])
        globals()["df_{}".format(i)] = pd.DataFrame(globals()["df_{}".format(i)])
        globals()["df_{}".format(i)].index = train_scdf.index
#         globals()["df_{}".format(i)].columns = [i]

    return globals()["df_{}".format(i)]

# shift data
def shift_df(*dataframes, start=1, end=11):  
    shift_num = end
    for dataframe in dataframes:
        for s in range(start, end):
            dataframe['shift_{}'.format(s)] = dataframe[dataframe.columns[0]].shift(s)
        dataframe.dropna(inplace=True)
    return dataframe

# concat dataframe
def df_concat(*dataframes):
    all_Df =  pd.concat([*dataframes], axis=1)
#     all_Df.dropna(inplace=True)
    return all_Df

# drop original values
def drop_ori_cols(all_df):
    col_list = []
    for i in all_df.columns:
        if 'shift' in i:
            pass
        elif 'Close' in i:
            pass
        else:
            col_list.append(i)
    all_df.drop(col_list, axis=1, inplace=True)
    
    return all_df

# set y target & split train test
def set_split(all_df, num=1, xnum=10):
    xdata = all_df.drop(1, axis=1)[:-num]
    ydata = all_df[1][num:].values
    
    x_train = xdata[:-xnum]
    x_test = xdata[-xnum:]

    ytrain = ydata[:-xnum]
    ytest = ydata[-xnum:]
    
    shift_num = 1
    x_train = np.array(x_train).reshape(len(x_train), int(len(x_train.columns)/shift_num), shift_num)
    x_test = np.array(x_test).reshape(len(x_test), int(len(x_test.columns)/shift_num), shift_num)

    
    return x_train, x_test, ytrain, ytest

# make LSTM model
def LSTMmodel_01():
    from tensorflow.keras.layers import LSTM 
    from tensorflow.keras.models import Sequential 
    from tensorflow.keras.layers import Dense 
    import tensorflow.keras.backend as K 
    from tensorflow.keras.callbacks import EarlyStopping


    model = Sequential() # Sequeatial Model 
    model.add(LSTM(20, input_shape=(x_train.shape[1], x_train.shape[2]))) # (timestep, feature) 
    
    model.add(Dense(1)) # output = 1 
    model.compile(loss='mean_squared_error', optimizer='adam') 
    model.summary()
    
    return model

# inverse predict 
def get_real(pred, scaler_Close):
    
    real = np.exp(scaler_Close.inverse_transform(pred))
    real = pd.DataFrame(real, columns = pred.columns)
    real['err'] = real['real'] - real['pred']
    
    return real

# model save
def save(model, name):
    # 전체 모델을 HDF5 파일로 저장합니다
    model.save('./src/parkjunsung/models/' + str(name) + '.h5')
    
# shift close
def future_stock(dataframe):
    for s in range(1, 2):
        dataframe['shift_{}'.format(s)] = dataframe[dataframe.columns[0]].shift(s)
    return dataframe

In [5]:
ticker = "AAPL"
api_key = "d2b83a9cbe59bd13f8c7615015e41b0e"

In [ ]:
# Show the available companies
companies = fa.available_companies(api_key)

# Collect general company information
profile = fa.profile(ticker, api_key)

# Collect recent company quotes
quotes = fa.quote(ticker, api_key)

# Collect market cap and enterprise value
entreprise_value = fa.enterprise(ticker, api_key)

# Show recommendations of Analysts
ratings = fa.rating(ticker, api_key)

# Obtain DCFs over time
dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# Collect the Balance Sheet statements
balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# Collect the Income Statements
income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# Collect the Cash Flow Statements
cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# Show Key Metrics
key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# Show a large set of in-depth ratios
financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# Show the growth of the company
growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# Download general stock data
stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# Download detailed stock data
stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")

In [95]:
companies

,name,price,exchange
symbol,,,
SPY,SPDR S&P 500,326.5200,NYSE Arca
CMCSA,Comcast Corporation Class A Common Stock,42.8000,Nasdaq Global Select
KMI,Kinder Morgan Inc.,14.1000,New York Stock Exchange
INTC,Intel Corporation,47.7300,Nasdaq Global Select
MU,Micron Technology Inc.,50.0550,Nasdaq Global Select
...,...,...,...
PSACU,PSACU,9.9600,NasdaqGS
SRGA,SRGA,2.8100,NasdaqGS
VWID,VWID,25.2962,BATS
